## IMPORTING REQUIRED LIBRARIES

In [373]:
from sklearn.cluster import KMeans

In [374]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes
import geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# <font color='blue'>Connecting to Foursquare API through Credentials</font>

In [375]:
CLIENT_ID = 'S4E3TQEW1GAPZZIHR0BLREOVZPQLCEHAYGJBM3T3Q0QHNI31' # your Foursquare ID
CLIENT_SECRET = 'QP3EVLQYLKGV4DGNHBTDGGIQ0JF5T11LPNZW2KWBBSQLE0N1' # your Foursquare Secret
VERSION = '20191225'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S4E3TQEW1GAPZZIHR0BLREOVZPQLCEHAYGJBM3T3Q0QHNI31
CLIENT_SECRET:QP3EVLQYLKGV4DGNHBTDGGIQ0JF5T11LPNZW2KWBBSQLE0N1


###   <font color='red'>Fetching Results of places around Connaught Place Area in New Delhi from FourSquare API</font>

# <font color='green'><b>Search Query for Restaurants</b> </font>

In [378]:
search_query = 'Restaurant'
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [379]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S4E3TQEW1GAPZZIHR0BLREOVZPQLCEHAYGJBM3T3Q0QHNI31&client_secret=QP3EVLQYLKGV4DGNHBTDGGIQ0JF5T11LPNZW2KWBBSQLE0N1&ll=28.6289169,77.2206152&v=20191225&query=Restaurant&radius=5000&limit=30'

# Results in Json Objects

In [ ]:
results = requests.get(url).json()
results

# Converting into Data Frame

In [382]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(5)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,5177f8d5e4b0f361220d4766,Anand Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1577792969,False,35 Community Centre,New Friends Colony,28.632091,77.224958,"[{'label': 'display', 'lat': 28.63209146656240...",552,110025,IN,New Delhi,Delhi,India,"[35 Community Centre (New Friends Colony), New...",NaN
1,525ad0c511d2bc4d09aca87d,Anand Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1577792969,False,NaN,NaN,28.628908,77.220901,"[{'label': 'display', 'lat': 28.62890815734863...",27,NaN,IN,NaN,NaN,India,[India],NaN
2,4b79606ef964a52089f62ee3,Kwality Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1577792969,False,Connaught Place,NaN,28.629744,77.217431,"[{'label': 'display', 'lat': 28.62974364503556...",324,NaN,IN,New Delhi,Delhi,India,"[Connaught Place, New Delhi, Delhi, India]",NaN
3,4f5063bbe4b062d9d9d5a175,Tom Yam Rooftop Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1577792969,False,Hare Krishna hotel,NaN,28.641011,77.212043,"[{'label': 'display', 'lat': 28.641011, 'lng':...",1585,NaN,IN,New Delhi,Delhi,India,"[Hare Krishna hotel, New Delhi, Delhi, India]",NaN
4,4d4d658b13d88cfa9b7a08da,Parikrama - The Revolving Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1577792969,False,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",NaN,28.628773,77.222949,"[{'label': 'display', 'lat': 28.62877338737562...",228,110001,IN,New Delhi,Delhi,India,"[22, Antariksh Bhavan, Kasturba Gandhi Marg, C...",NaN


In [383]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(5)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Anand Restaurant,Indian Restaurant,35 Community Centre,New Friends Colony,28.632091,77.224958,"[{'label': 'display', 'lat': 28.63209146656240...",552,110025,IN,New Delhi,Delhi,India,"[35 Community Centre (New Friends Colony), New...",5177f8d5e4b0f361220d4766
1,Anand Restaurant,Indian Restaurant,NaN,NaN,28.628908,77.220901,"[{'label': 'display', 'lat': 28.62890815734863...",27,NaN,IN,NaN,NaN,India,[India],525ad0c511d2bc4d09aca87d
2,Kwality Restaurant,Indian Restaurant,Connaught Place,NaN,28.629744,77.217431,"[{'label': 'display', 'lat': 28.62974364503556...",324,NaN,IN,New Delhi,Delhi,India,"[Connaught Place, New Delhi, Delhi, India]",4b79606ef964a52089f62ee3
3,Tom Yam Rooftop Restaurant,Thai Restaurant,Hare Krishna hotel,NaN,28.641011,77.212043,"[{'label': 'display', 'lat': 28.641011, 'lng':...",1585,NaN,IN,New Delhi,Delhi,India,"[Hare Krishna hotel, New Delhi, Delhi, India]",4f5063bbe4b062d9d9d5a175
4,Parikrama - The Revolving Restaurant,Indian Restaurant,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",NaN,28.628773,77.222949,"[{'label': 'display', 'lat': 28.62877338737562...",228,110001,IN,New Delhi,Delhi,India,"[22, Antariksh Bhavan, Kasturba Gandhi Marg, C...",4d4d658b13d88cfa9b7a08da


# Names of Restaurants

In [384]:
dataframe_filtered.name.head()

0                        Anand Restaurant
1                        Anand Restaurant
2                      Kwality Restaurant
3              Tom Yam Rooftop Restaurant
4    Parikrama - The Revolving Restaurant
Name: name, dtype: object

# <i> Map of places plotted Using Foursquare API</i>

In [439]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the location

# add a red circle marker to represent the Conrad Hotel
folium.Map(
    [latitude, longitude],
    radius=1000,
    color='red',
    popup='Tea',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    

# display map
venues_map


In [438]:
venues_map.render()

In [386]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# <font color='blue'>Storing into new Data Frame</font>

In [387]:
dataframe=dataframe_filtered

In [388]:
dataframe['Locality']='Connaught Place'

In [389]:
df=dataframe[['name','Locality','categories','address','lat','lng',]]

In [390]:
df.head()

,name,Locality,categories,address,lat,lng
0,Anand Restaurant,Connaught Place,Indian Restaurant,35 Community Centre,28.632091,77.224958
1,Anand Restaurant,Connaught Place,Indian Restaurant,NaN,28.628908,77.220901
2,Kwality Restaurant,Connaught Place,Indian Restaurant,Connaught Place,28.629744,77.217431
3,Tom Yam Rooftop Restaurant,Connaught Place,Thai Restaurant,Hare Krishna hotel,28.641011,77.212043
4,Parikrama - The Revolving Restaurant,Connaught Place,Indian Restaurant,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",28.628773,77.222949


In [391]:
df['ratings']=[4,4.3,4.5,3,4,2,2.8,4.6,4.3,3,3.8,3.6,4.2,4,3.9,2.8,2,3.9,4.6,4,4,4,4,4,4,4,4,4,3.9,3]

c:\users\acer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [392]:
df.head()

,name,Locality,categories,address,lat,lng,ratings
0,Anand Restaurant,Connaught Place,Indian Restaurant,35 Community Centre,28.632091,77.224958,4.0
1,Anand Restaurant,Connaught Place,Indian Restaurant,NaN,28.628908,77.220901,4.3
2,Kwality Restaurant,Connaught Place,Indian Restaurant,Connaught Place,28.629744,77.217431,4.5
3,Tom Yam Rooftop Restaurant,Connaught Place,Thai Restaurant,Hare Krishna hotel,28.641011,77.212043,3.0
4,Parikrama - The Revolving Restaurant,Connaught Place,Indian Restaurant,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",28.628773,77.222949,4.0


# <font color='Orange'>Collected data from kaggle dataset about Restaurants in New Delhi and this data converted into Data Frame</font>

In [393]:
import xlrd
import pandas as pd
import numpy as np

In [394]:
df_2=pd.read_excel(r"C:\Users\acer\Desktop\naglong1396.xlsx")

In [395]:
df_2.head()

,Restaurant ID,Restaurant Name,Country Code,,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,300397.0,Subway,1.0,New Delhi,"4, F-115, Main Market, Rajouri Garden, New Delhi",Rajouri Garden,"Rajouri Garden, New Delhi",77.121688,28.644926,American,500.0,Indian Rupees(Rs.),No,Yes,No,No,2.0,2.7,Orange,Average,114.0
1,306247.0,Subway,1.0,New Delhi,"E-25A, Hauz Khas Market, Hauz Khas, New Delhi",Hauz Khas,"Hauz Khas, New Delhi",77.208147,28.551015,American,500.0,Indian Rupees(Rs.),No,Yes,No,No,2.0,2.8,Orange,Average,87.0
2,301190.0,Subway,1.0,New Delhi,"1/3, E Block, Main Road, Malviya Nagar, New Delhi",Malviya Nagar,"Malviya Nagar, New Delhi",77.209510,28.534046,American,500.0,Indian Rupees(Rs.),No,Yes,No,No,2.0,2.8,Orange,Average,183.0
3,7322.0,Subway,1.0,New Delhi,"A-1, Arjun Nagar, Safdarjung, New Delhi",Safdarjung,"Safdarjung, New Delhi",77.200704,28.561644,American,500.0,Indian Rupees(Rs.),No,Yes,No,No,2.0,2.9,Orange,Average,188.0
4,18449796.0,Big Fat Sandwich,1.0,New Delhi,"A-15A Front, Hauz Khas, New Delhi",Hauz Khas,"Hauz Khas, New Delhi",77.203932,28.550327,American,800.0,Indian Rupees(Rs.),No,No,No,No,2.0,2.9,Orange,Average,21.0


# Retrieving Name,Locality,Address,Cuisine,Location,Rating of each Restaurant

In [396]:
df_excel=df_2[['Restaurant Name','Locality','Address','Cuisines','Latitude','Longitude','Aggregate rating']]

In [397]:
df_excel.head()

,Restaurant Name,Locality,Address,Cuisines,Latitude,Longitude,Aggregate rating
0,Subway,Rajouri Garden,"4, F-115, Main Market, Rajouri Garden, New Delhi",American,28.644926,77.121688,2.7
1,Subway,Hauz Khas,"E-25A, Hauz Khas Market, Hauz Khas, New Delhi",American,28.551015,77.208147,2.8
2,Subway,Malviya Nagar,"1/3, E Block, Main Road, Malviya Nagar, New Delhi",American,28.534046,77.209510,2.8
3,Subway,Safdarjung,"A-1, Arjun Nagar, Safdarjung, New Delhi",American,28.561644,77.200704,2.9
4,Big Fat Sandwich,Hauz Khas,"A-15A Front, Hauz Khas, New Delhi",American,28.550327,77.203932,2.9


In [398]:
df_excel.columns=['Name','Locality','Address','Cuisines','Latitude','Longitude','Ratings']
df_excel.head()

,Name,Locality,Address,Cuisines,Latitude,Longitude,Ratings
0,Subway,Rajouri Garden,"4, F-115, Main Market, Rajouri Garden, New Delhi",American,28.644926,77.121688,2.7
1,Subway,Hauz Khas,"E-25A, Hauz Khas Market, Hauz Khas, New Delhi",American,28.551015,77.208147,2.8
2,Subway,Malviya Nagar,"1/3, E Block, Main Road, Malviya Nagar, New Delhi",American,28.534046,77.209510,2.8
3,Subway,Safdarjung,"A-1, Arjun Nagar, Safdarjung, New Delhi",American,28.561644,77.200704,2.9
4,Big Fat Sandwich,Hauz Khas,"A-15A Front, Hauz Khas, New Delhi",American,28.550327,77.203932,2.9


In [399]:
df.columns=['Name','Locality','Cuisines','Address','Latitude','Longitude','Ratings']
df.head()

,Name,Locality,Cuisines,Address,Latitude,Longitude,Ratings
0,Anand Restaurant,Connaught Place,Indian Restaurant,35 Community Centre,28.632091,77.224958,4.0
1,Anand Restaurant,Connaught Place,Indian Restaurant,NaN,28.628908,77.220901,4.3
2,Kwality Restaurant,Connaught Place,Indian Restaurant,Connaught Place,28.629744,77.217431,4.5
3,Tom Yam Rooftop Restaurant,Connaught Place,Thai Restaurant,Hare Krishna hotel,28.641011,77.212043,3.0
4,Parikrama - The Revolving Restaurant,Connaught Place,Indian Restaurant,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",28.628773,77.222949,4.0


# <font color='Orange'>Merging both Foresquare,Kaggle data sets</font>

In [400]:
concat=pd.concat([df,df_excel],axis=0)

c:\users\acer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [401]:
concat.shape

(1426, 7)

In [402]:
concat.reset_index(drop=True, inplace=True)

## <font color='Orange'>Printing First 10 Restaurant names using head function</font>

In [403]:
concat['Name'].head(10)

0                        Anand Restaurant
1                        Anand Restaurant
2                      Kwality Restaurant
3              Tom Yam Rooftop Restaurant
4    Parikrama - The Revolving Restaurant
5                       Gulati Restaurant
6                     The Host Restaurant
7                Japanese Fuji Restaurant
8                     anand restaurant cp
9                          KFC Restaurant
Name: Name, dtype: object

# Concatenated DataFrame

In [404]:
concat.head()

,Address,Cuisines,Latitude,Locality,Longitude,Name,Ratings
0,35 Community Centre,Indian Restaurant,28.632091,Connaught Place,77.224958,Anand Restaurant,4.0
1,NaN,Indian Restaurant,28.628908,Connaught Place,77.220901,Anand Restaurant,4.3
2,Connaught Place,Indian Restaurant,28.629744,Connaught Place,77.217431,Kwality Restaurant,4.5
3,Hare Krishna hotel,Thai Restaurant,28.641011,Connaught Place,77.212043,Tom Yam Rooftop Restaurant,3.0
4,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",Indian Restaurant,28.628773,Connaught Place,77.222949,Parikrama - The Revolving Restaurant,4.0


## Removing null values in dataframe

In [405]:
concat.dropna(inplace=True)

In [406]:
concat.head(20)

,Address,Cuisines,Latitude,Locality,Longitude,Name,Ratings
0,35 Community Centre,Indian Restaurant,28.632091,Connaught Place,77.224958,Anand Restaurant,4.0
2,Connaught Place,Indian Restaurant,28.629744,Connaught Place,77.217431,Kwality Restaurant,4.5
3,Hare Krishna hotel,Thai Restaurant,28.641011,Connaught Place,77.212043,Tom Yam Rooftop Restaurant,3.0
4,"22, Antariksh Bhavan, Kasturba Gandhi Marg, Co...",Indian Restaurant,28.628773,Connaught Place,77.222949,Parikrama - The Revolving Restaurant,4.0
5,Pandara Road,Indian Restaurant,28.608010,Connaught Place,77.229989,Gulati Restaurant,2.0
7,"Speedbird house, M Block, Middle circle, Conna...",Japanese Restaurant,28.633099,Connaught Place,77.222122,Japanese Fuji Restaurant,4.6
9,"A Block, Inner Circle",Fast Food Restaurant,28.633158,Connaught Place,77.218201,KFC Restaurant,3.0
13,H 27,Indian Restaurant,28.635210,Connaught Place,77.218079,Veda Restaurant,4.0
14,M Block,Indian Restaurant,28.632262,Connaught Place,77.219214,York's Restaurant,3.9
16,"G-68,Outer Circle, Connaught Place, New Delhi",Restaurant,28.631052,Connaught Place,77.216243,My Lounge Bar & Restaurant,2.0


# Resetting index values after removing null values

In [407]:
concat.reset_index(drop=True,inplace=True)

In [408]:
concat.index=concat.index+1

# Visualising locations of dataset through Folium

In [440]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the location

# add a red circle marker to represent the Conrad Hotel
folium.Map(
    [latitude, longitude],
    radius=1000,
    color='red',
    popup='Tea',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(concat.Latitude, concat.Longitude,concat.Name):
    folium.Circle(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    

# display map
venues_map1

In [ ]:
venues_map1.render()

In [410]:
concat3=concat.drop(columns=['Name','Address'])

In [411]:
concat4=concat3
concat4.head()

,Cuisines,Latitude,Locality,Longitude,Ratings
1,Indian Restaurant,28.632091,Connaught Place,77.224958,4.0
2,Indian Restaurant,28.629744,Connaught Place,77.217431,4.5
3,Thai Restaurant,28.641011,Connaught Place,77.212043,3.0
4,Indian Restaurant,28.628773,Connaught Place,77.222949,4.0
5,Indian Restaurant,28.608010,Connaught Place,77.229989,2.0


# <font color='red'>One Hot Encoding of Locality,Cuisines columns in data frame</font>

In [412]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()

In [413]:
a=pd.get_dummies(concat4.Locality)
#concat4
a
b=pd.get_dummies(concat4.Cuisines)
b
b=b.join(a)


In [414]:
concat4['Cuisines'].unique()


array(['Indian Restaurant', 'Thai Restaurant', 'Japanese Restaurant',
       'Fast Food Restaurant', 'Restaurant', 'Chinese Restaurant',
       'Moroccan Restaurant', 'Bar', 'Mexican Restaurant', 'Karaoke Bar',
       'American', 'Bakery', 'Bakery, Desserts, Fast Food', 'Bengali',
       'Others', 'Beverages', 'Cafe', 'Cafe,Bakery,Others', 'Chinese',
       'Continental', 'Desserts', 'Fast Food', 'Ice Cream', 'Italian',
       'Mix', 'North Indian', 'North Indian, Chinese', 'South Indian',
       'Street Food'], dtype=object)

# Adding latitude,longitude columns to onehotencoded dataframe

In [415]:
b['Latitude']=concat4['Latitude']
b['Longitude']=concat4['Longitude']
b['Ratings']=concat4['Ratings']
#,'Longitude','Ratings']=concat4['Latitude','Longitude','Ratings']

In [416]:
b.head()

,American,Bakery,"Bakery, Desserts, Fast Food",Bar,Bengali,Beverages,Cafe,"Cafe,Bakery,Others",Chinese,Chinese Restaurant,Continental,Desserts,Fast Food,Fast Food Restaurant,Ice Cream,Indian Restaurant,Italian,Japanese Restaurant,Karaoke Bar,Mexican Restaurant,Mix,Moroccan Restaurant,North Indian,"North Indian, Chinese",Others,Restaurant,South Indian,Street Food,Thai Restaurant,Chandni Chowk,Connaught Place,Defence Colony,Delhi University-GTB Nagar,Hauz Khas,Kalkaji,Karol Bagh,Krishna Nagar,Mahipalpur,Malviya Nagar,Mayur Vihar Phase 1,Mukherjee Nagar,Pitampura,Rajinder Nagar,Rajouri Garden,Safdarjung,Satyaniketan,Shahdara,Latitude,Longitude,Ratings
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.632091,77.224958,4.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.629744,77.217431,4.5
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.641011,77.212043,3.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.628773,77.222949,4.0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.608010,77.229989,2.0


# <font color='cyan'>Applying K-means Clustering with 5 neighbors</font>

In [417]:
# set number of clusters
kclusters = 3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, max_iter=1000).fit(b)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:1000] 
print('Centroids:', kmeans.cluster_centers_)


Centroids: [[ 1.78571429e-02  2.67857143e-02  6.47321429e-02  1.11607143e-03
   1.11607143e-03  1.11607143e-02  3.23660714e-02  7.14285714e-02
   1.42857143e-01  1.11607143e-03  5.02232143e-02  2.12053571e-02
   2.23214286e-01  1.11607143e-03  4.35267857e-02  8.92857143e-03
   2.23214286e-02  1.11607143e-03  1.11607143e-03  1.11607143e-03
   4.46428571e-03  1.11607143e-03 -3.41393580e-15  1.97064587e-15
   1.38392857e-01  1.11607143e-03  5.35714286e-02  5.58035714e-02
   1.11607143e-03  4.46428571e-02  1.01562500e-01  6.25000000e-02
   5.02232143e-02  5.80357143e-02  5.02232143e-02  5.24553571e-02
   4.01785714e-02  2.56696429e-02  6.25000000e-02  4.79910714e-02
   3.57142857e-02  5.91517857e-02  5.80357143e-02  6.80803571e-02
   5.35714286e-02  7.25446429e-02  5.69196429e-02  2.86199866e+01
   7.72076374e+01  3.37700893e+00]
 [-7.28583860e-17 -5.89805982e-17 -1.24900090e-16 -4.55364912e-18
  -4.55364912e-18  2.86229374e-17 -6.24500451e-17 -2.91433544e-16
  -5.82867088e-16 -4.55364912e

# Assigning labels to Data Frame

In [418]:
b['labels']=kmeans.labels_
b.head()

,American,Bakery,"Bakery, Desserts, Fast Food",Bar,Bengali,Beverages,Cafe,"Cafe,Bakery,Others",Chinese,Chinese Restaurant,Continental,Desserts,Fast Food,Fast Food Restaurant,Ice Cream,Indian Restaurant,Italian,Japanese Restaurant,Karaoke Bar,Mexican Restaurant,Mix,Moroccan Restaurant,North Indian,"North Indian, Chinese",Others,Restaurant,South Indian,Street Food,Thai Restaurant,Chandni Chowk,Connaught Place,Defence Colony,Delhi University-GTB Nagar,Hauz Khas,Kalkaji,Karol Bagh,Krishna Nagar,Mahipalpur,Malviya Nagar,Mayur Vihar Phase 1,Mukherjee Nagar,Pitampura,Rajinder Nagar,Rajouri Garden,Safdarjung,Satyaniketan,Shahdara,Latitude,Longitude,Ratings,labels
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.632091,77.224958,4.0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.629744,77.217431,4.5,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.641011,77.212043,3.0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.628773,77.222949,4.0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.608010,77.229989,2.0,0


# <font color='Orange'> Visualising Clusters in Map for Restaurants in New Delhi city </font>

In [419]:
import matplotlib.cm as cm
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,label in zip(b['Latitude'], b['Longitude'],b['labels']):
    
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if label==0:
        color='green'
    elif label==1:
        color='blue'
    elif label==2:
        color='black'
    elif label==3:
        color='yellow'
    else:
        color='white'
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Anlaysis of cluster1(label 0)

In [420]:
cluster1= b[b['labels']==0]

In [421]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
cluster1 = cluster1.loc[:, (cluster1 != 0).any(axis=0)]

# Deleting columns with all zero values

In [422]:
cluster1 = cluster1.loc[:, (cluster1 != 0).any(axis=0)]
k=list(cluster1.columns)

# Finding out main cuisines,locations,Rating in cluster1

In [423]:
for i in k:
    if(cluster1[i].sum()==0):
        cluster1.drop(i)
    print(cluster1[i].sum())
    print(i)

16
American
24
Bakery
58
Bakery, Desserts, Fast Food
1
Bar
1
Bengali
10
Beverages
29
Cafe
64
Cafe,Bakery,Others
128
Chinese
1
Chinese Restaurant
45
Continental
19
Desserts
200
Fast Food
1
Fast Food Restaurant
39
Ice Cream
8
Indian Restaurant
20
Italian
1
Japanese Restaurant
1
Karaoke Bar
1
Mexican Restaurant
4
Mix
1
Moroccan Restaurant
124
Others
1
Restaurant
48
South Indian
50
Street Food
1
Thai Restaurant
40
Chandni Chowk
91
Connaught Place
56
Defence Colony
45
Delhi University-GTB Nagar
52
Hauz Khas
45
Kalkaji
47
Karol Bagh
36
Krishna Nagar
23
Mahipalpur
56
Malviya Nagar
43
Mayur Vihar Phase 1
32
Mukherjee Nagar
53
Pitampura
52
Rajinder Nagar
61
Rajouri Garden
48
Safdarjung
65
Satyaniketan
51
Shahdara
25643.507983742864
Latitude
69178.04308354718
Longitude
3025.7999999999997
Ratings


In [424]:
len(cluster1)

896

# Minimum and Maximum Range of Ratings in cluster1

In [425]:
print(cluster1.Ratings.min())
cluster1.Ratings.max()

2.0


4.9

# Anlaysis of cluster2(label 1)

In [426]:
cluster2= b[b['labels']==1]
cluster2 = cluster2.loc[:, (cluster2 != 0).any(axis=0)]

In [427]:
l=cluster2.columns

# Finding out main cuisines,locations,Rating in cluster2

In [428]:
for i in l:
    print(i)
    if(cluster2[i].sum()==0):
        cluster2.drop(i)
    print(cluster2[i].sum())
    i

North Indian
306
Chandni Chowk
23
Connaught Place
33
Defence Colony
17
Delhi University-GTB Nagar
8
Hauz Khas
12
Kalkaji
17
Karol Bagh
12
Krishna Nagar
23
Mahipalpur
23
Malviya Nagar
8
Mayur Vihar Phase 1
17
Mukherjee Nagar
26
Pitampura
18
Rajinder Nagar
13
Rajouri Garden
22
Safdarjung
11
Satyaniketan
3
Shahdara
20
Latitude
8760.25550186
Longitude
23626.71613131
Ratings
1001.7
labels
306


In [429]:
len(cluster2)

306

# Minimum and Maximum Range of Ratings in cluster2

In [430]:
print(cluster2.Ratings.min())
cluster2.Ratings.max()

2.4


4.8

# Anlaysis of cluster3(label 2)

In [431]:
cluster3= b[b['labels']==2]
cluster3 = cluster3.loc[:, (cluster3 != 0).any(axis=0)]

In [432]:
l=cluster3.columns

In [433]:
len(cluster3)

210

# Finding out main cuisines,locations,Rating in cluster3

In [434]:
for i in l:
    print(i)
    if(cluster3[i].sum()==0):
        cluster3.drop(i)
    print(cluster3[i].sum())
    i

North Indian, Chinese
210
Chandni Chowk
2
Connaught Place
15
Defence Colony
12
Delhi University-GTB Nagar
12
Hauz Khas
7
Kalkaji
9
Karol Bagh
16
Krishna Nagar
8
Mahipalpur
20
Malviya Nagar
20
Mayur Vihar Phase 1
13
Mukherjee Nagar
7
Pitampura
12
Rajinder Nagar
14
Rajouri Garden
15
Safdarjung
11
Satyaniketan
7
Shahdara
10
Latitude
6009.09780369
Longitude
16212.383034619997
Ratings
668.5999999999999
labels
420


In [435]:
print(cluster3.Ratings.min())
cluster3.Ratings.max()

2.0


4.4